<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_ML_20210602/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 3s 0us/step


In [3]:
import os
import glob
dn = os.path.split(dataset)[0]
fn = os.path.join(dn, "aclImdb", "train", "pos", "*")
# print(fn)
fns = glob.glob(fn)
with open(fns[0], "r", encoding="utf-8") as f:
  print(f.read())

The Perfect Son is a story about two 30-something brothers, one who is seemingly "perfect" and the other who is basically a screw-up, frequently landing himself in drug rehab centers. After the death of their father, the two are brought together after a long absence and the usual sibling rivalry resurfaces. It isn't until the "perfect" brother makes the startling revelation that he has AIDS that the irresponsible younger brother finally makes a move to get his life in order, and take some responsibility.<br /><br />The movie does a nice job of chronicling the younger brother's "comeback", though it may seem a bit far-fetched at times (beating drug addiction is never so easy). What makes the film more tender is the treatment of AIDS, a topic that has become somewhat passe in cinema over the last 5-10 years. And also the development of an almost sweet relationship between the two formerly feuding brothers is very believable and well-done. The two main actors were both very competent, if 

In [4]:
import pandas as pd
def get_data(base):
  contents, target = [], []
  dn = os.path.split(dataset)[0]
  fn = os.path.join(dn, "aclImdb", base, "pos", "*.txt")
  fns = glob.glob(fn)
  for p in fns:
    with open(p, "r", encoding="utf-8") as f:
      contents.append(f.read())
      target.append(1)
  fn = os.path.join(dn, "aclImdb", base, "neg", "*.txt")
  fns = glob.glob(fn)
  for p in fns:
    with open(p, "r", encoding="utf-8") as f:
      contents.append(f.read())
      target.append(0)
  df = pd.DataFrame({
      "contents":contents,
      "ans":target
  })
  return df
train_df = get_data(base="train")
test_df = get_data(base="test")

In [5]:
# Step1 : Tokenize
# 我們絕對不會使用0. 因為0是拿來做padding
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["contents"])

In [6]:
# Step2. Sequenize
# tok.word_index
# tok.index_word
# 文章(token) -> sequence
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,1,400,489,6,3,62,41,104,1085.0,139.0,1090.0,28.0,34.0,6.0,1570.0,400.0,2.0,1.0,82.0,34.0,6.0,688.0,3.0,53.0,306.0,8.0,1389.0,100.0,1.0,338.0,4.0,65.0,333.0,1.0,104.0,23.0,835.0,292.0,100.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1504,339,43,4,467,378,7,7,38.0,310.0,2.0,91.0,1893.0,56.0,269.0,130.0,2.0,47.0,2.0,92.0,268.0,8.0,24.0,2.0,22.0,76.0,1.0,428.0,157.0,247.0,12.0,104.0,2847.0,2.0,419.0,846.0,26.0,24.0,104.0,2847.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,6,3,17,12,44,3,173,4.0,180.0,12.0,61.0,857.0,81.0,67.0,388.0,57.0,70.0,47.0,23.0,46.0,180.0,12.0,23.0,537.0,1951.0,635.0,39.0,58.0,479.0,6.0,12.0,9.0,800.0,5.0,2218.0,3.0,746.0,4.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,116,1,99,2,202,1,1,23.0,272.0,92.0,1.0,17.0,18.0,128.0,471.0,199.0,9.0,155.0,43.0,4.0,155.0,9.0,13.0,1187.0,45.0,1.0,99.0,185.0,1624.0,1187.0,10.0,59.0,25.0,38.0,2.0,10.0,241.0,665.0,329.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1180,1237,2016,2823,2096,10,13,63,176.0,1.0,83.0,55.0,10.0,293.0,11.0,17.0,10.0,380.0,48.0,3.0,1720.0,1237.0,392.0,555.0,161.0,6.0,1624.0,7.0,7.0,70.0,11.0,17.0,6.0,979.0,18.0,9.0,79.0,6.0,21.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1,17,13,3,391,19,1022,1,83.0,28.0,11.0,6.0,21.0,3.0,49.0,19.0,30.0,29.0,1.0,2866.0,68.0,539.0,18.0,1.0,62.0,13.0,1032.0,2.0,54.0,144.0,450.0,13.0,2168.0,53.0,8.0,9.0,8.0,1.0,83.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,161,7,7,282,7,7,111,2718,3.0,665.0,2419.0,1126.0,80.0,3.0,12.0,713.0,38.0,534.0,41.0,1.0,2450.0,4.0,81.0,12.0,68.0,553.0,31.0,1143.0,7.0,7.0,730.0,112.0,8.0,58.0,110.0,25.0,10.0,213.0,637.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,6,3,52,1029,19,10,283,63.0,249.0,48.0,6.0,13.0,41.0,46.0,1311.0,166.0,3.0,550.0,8.0,1.0,1408.0,12.0,33.0,29.0,261.0,6.0,46.0,240.0,4.0,2.0,818.0,163.0,95.0,29.0,8.0,65.0,202.0,93.0,649.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,722,825,1529,8,10,1385,3,84,599.0,8.0,1.0,1166.0,4.0,1.0,2226.0,966.0,509.0,10.0,165.0,928.0,5.0,3.0,1325.0,5.0,791.0,53.0,16.0,447.0,2278.0,5.0,131.0,1907.0,4.0,1248.0,45.0,61.0,1661.0,2135.0,97.0,719.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
SEQ_LEN = 512

In [8]:
# Stetp3. Padding
from  tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad  = pad_sequences(x_train_seq, maxlen=SEQ_LEN)
x_test_pad  = pad_sequences(x_test_seq, maxlen=SEQ_LEN)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,52,864,2,70,221,1,104,290,153,68,196,52,45,21,1898,7,7,3,1153,83,788,778,36,164,2,561,622,202,593,1127,4,1,274,6,1777,221,14,70,7,7
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1335,2,124,21,94,95,354,8,1,127,91,29,3,1359,2,175,12,31,40,30,29,131,81,40,1,279,135,10,116,146,105,19,228,1,877,6,304,1093,2,1906
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,197,1,2,1,28,12,269,37,3,191,1894,126,480,43,10,423,63,1,111,7,7,1321,344,324,2052,1,550,3,173,1,1866,2537,6,391,1033,2,1,357,6,592
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,39,124,21,37,369,39,22,121,40,1339,2137,45,10,97,1577,20,58,86,8,116,2,86,108,208,10,25,329,1,10,59,518,43,4,15,249,18,1395,47,6,207
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,824,15,24,778,8,11,17,7,7,35,134,227,36,400,404,32,1135,17,16,3,114,224,4,194,493,9,7,7,1321,344,339,454,43,4,467,287,3,647,39,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,976,13,438,441,11,19,215,287,2751,2,1,343,17,156,158,78,230,15,69,39,58,319,2,72,78,1131,448,446,70,276,1,372,28,77,27,125,2496,827,73,430
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17,1167,443,1222,105,503,212,25,63,74,5,213,53,16,3,17,37,11,7,7,58,672,297,43,4,155,45,9,68,53,5,69,11,17,59,76,1,672,4,1563
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,527,7,7,48,13,1,746,241,10,109,9,13,40,96,1029,48,568,5,135,124,1255,1,550,12,90,424,54,278,1147,67,291,335,69,388,11,944,4,3,19
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,122,100,3,80,3,2044,39,2357,3,77,3,177,1,4,32,1962,831,1712,15,2,355,1,4,1454,1329,7,7,29,8,29,1,580,4,3,1445,8,1,225,69


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
layers = [
    # 每個詞彙都有128個係數, 3001 * 128 = 384128
    # !!!tokenize 有多少個可調(上面要記得調): 3000 -> 4000
    # !!!詞向量維度可調(根據問題調, 簡單小/困難大): 128 -> 256
    # !!!input_length(上面要記得調) : 512 -> 256
    Embedding(3001, 128, mask_zero=True, input_length=SEQ_LEN),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [10]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
        optimizer=Adam(),
        metrics=["accuracy"])

In [11]:
import numpy as np
y_train = np.array(train_df["ans"])
y_test = np.array(test_df["ans"])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# validation_split : 切出一些驗證資料來做驗證(0.1:54000筆資料訓練)
# epochs: 看幾輪(10輪: 54000 * 10)
# batch_size: 看了多少筆再做一次梯度調整(看圖片大小, 圖片小:200, 圖片大:20)
# 200batch, 10epochs: 54000 * 10 / 200 -> 2700次(梯度調整)
# verbose: 0(無聲) 1(最詳細) 2(沒有進度條)
callback = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("imdb.h5", save_best_only=True)
]
model.fit(x_train_pad,
      y_train,
      validation_split=0.1,
      epochs=100,
      batch_size=200,
      verbose=2,
      callbacks=callback)

Epoch 1/100
113/113 - 7s - loss: 0.6346 - accuracy: 0.6431 - val_loss: 0.6416 - val_accuracy: 0.6340
Epoch 2/100
113/113 - 6s - loss: 0.4667 - accuracy: 0.8274 - val_loss: 0.4419 - val_accuracy: 0.8200
Epoch 3/100
113/113 - 6s - loss: 0.3631 - accuracy: 0.8652 - val_loss: 0.3767 - val_accuracy: 0.8408
Epoch 4/100
113/113 - 8s - loss: 0.3127 - accuracy: 0.8820 - val_loss: 0.3503 - val_accuracy: 0.8484
Epoch 5/100
113/113 - 10s - loss: 0.2834 - accuracy: 0.8921 - val_loss: 0.3354 - val_accuracy: 0.8516
Epoch 6/100
113/113 - 9s - loss: 0.2639 - accuracy: 0.8985 - val_loss: 0.3229 - val_accuracy: 0.8576
Epoch 7/100
113/113 - 8s - loss: 0.2494 - accuracy: 0.9039 - val_loss: 0.2947 - val_accuracy: 0.8772
Epoch 8/100
113/113 - 8s - loss: 0.2390 - accuracy: 0.9087 - val_loss: 0.3177 - val_accuracy: 0.8636
Epoch 9/100
113/113 - 10s - loss: 0.2302 - accuracy: 0.9114 - val_loss: 0.3579 - val_accuracy: 0.8476
Epoch 10/100
113/113 - 8s - loss: 0.2233 - accuracy: 0.9142 - val_loss: 0.3611 - val_accu

In [13]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.2919 - accuracy: 0.8810


[0.2919420599937439, 0.8809999823570251]

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
layers = [
    # 每個詞彙都有128個係數, 3001 * 128 = 384128
    # !!!tokenize 有多少個可調(上面要記得調): 3000 -> 4000
    # !!!詞向量維度可調(根據問題調, 簡單小/困難大): 128 -> 256
    # !!!input_length(上面要記得調) : 512 -> 256
    Embedding(3001, 128, mask_zero=True)
]
w2v = Sequential(layers)
w2v.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         384128    
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [15]:
w = model.layers[0].get_weights()
w2v.layers[0].set_weights(w)

In [26]:
from scipy.spatial.distance import cosine
from numpy.linalg import norm
tok1 = tok.word_index["scary"]
word1 = [[tok1]]
v1 = w2v.predict(word1)[0][0]
tok2 = tok.word_index["horror"]
word2 = [[tok2]]
v2 = w2v.predict(word2)[0][0]
# 相似度 scipy : 1-cos -> 1-(1-cos) 只想要 cos 的方法
# cos: -1反向相似 / 0不相干 / 1正面相似, 0.5以上蠻像的兩個詞
print("Cos:",1 - cosine(v1,v2))
print("Cos:",np.dot(v1, v2) / (norm(v1)*norm(v2)))

Cos: 0.7462731003761292
Cos: 0.74627316
